# Sprint
## ディープラーニングフレームワーク1

## 1.このSprintについて

### Sprintの目的
- フレームワークのコードを読めるようにする
- フレームワークを習得し続けられるようになる
- 理論を知っている範囲をフレームワークで動かす

### どのように学ぶか
TensorFLowのサンプルコードを元に、これまで扱ってきたデータセットを学習していきます。

## 2.コードリーディング

TensorFLowによって2値分類を行うサンプルコードを載せました。今回はこれをベースにして進めます。


tf.kerasやtf.estimatorなどの高レベルAPIは使用していません。低レベルなところから見ていくことにします。

### 【問題1】
#### スクラッチを振り返る
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。


（例）


- 重みを初期化する必要があった
- エポックのループが必要だった

それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう。

- forward, backwardの実装が必要
- 最適化手法の実装が必要

### データセットの用意
以前から使用しているIrisデータセットを使用します。以下のサンプルコードでは`Iris.csv`が同じ階層にある想定です。


[Iris Species](https://www.kaggle.com/uciml/iris/data)


目的変数は`Species`ですが、3種類ある中から以下の2種類のみを取り出して使用します。

```
Iris-versicolor
Iris-virginica
```

### 【問題2】
#### スクラッチとTensorFlowの対応を考える
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。


それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。

In [75]:
#《サンプルコード》
# ＊バージョン1.5から1.14の間で動作を確認済みです。
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [76]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [7]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 22.2805, val_loss : 58.9261, acc : 0.750, val_acc : 0.375
Epoch 1, loss : 22.2832, val_loss : 19.2108, acc : 0.500, val_acc : 0.625
Epoch 2, loss : 3.3121, val_loss : 16.9526, acc : 0.750, val_acc : 0.375
Epoch 3, loss : 5.7934, val_loss : 4.0721, acc : 0.750, val_acc : 0.750
Epoch 4, loss : 0.0000, val_loss : 4.0555, acc : 1.000, val_acc : 0.688
Epoch 5, loss : 0.0001, val_loss : 2.2525, acc : 1.000, val_acc : 0.812
Epoch 6, loss : 0.0000, val_loss : 2.3361, acc : 1.000, val_acc : 0.812
Epoch 7, loss : 0.0000, val_loss : 1.5640, acc : 1.000, val_acc : 0.812
Epoch 8, loss : 0.0000, val_loss : 2.3751, acc : 1.000, val_acc : 0.812
Epoch 9, loss : 0.0024, val_loss : 1.1672, acc : 1.000, val_acc : 0.938
test_acc : 0.800


- 重み・バイアスの初期化

サンプルコードでは`tf.Variable`型で管理を行っている。

sessionを開始した際に一度だけ`tf.global_variables_initializer()`を実行することで、Variable（重みやバイアス）を初期化している。

- エポックのループ

スクラッチ実装の際と同じようにforループを使って実装している。

高レベルAPIである`tf.estimator`を使う場合だと`num_epochs`引数で指定できるらしい。

- forward, backward
- 最適化手法

最適化手法は実装済みで、損失関数をminimizeするように指定することでweightsとbiasesが最適化される。

実際にsession内でrunする際には、計算グラフの目的地点である`train_op`や`accracy`などに対して行うだけで、それより前の計算グラフは自動的に実行される。

スクラッチ実装の時のようにforループを使って出力を保存しながらレイヤーでの計算を行う必要はない。


## 3.他のデータセットへの適用

これまで扱ってきた小さなデータセットがいくつかあります。上記サンプルコードを書き換え、これらに対して学習・推定を行うニューラルネットワークを作成してください。


- Iris（3種類全ての目的変数を使用）
- House Prices

どのデータセットもtrain, val, testの3種類に分けて使用してください。

### 【問題3】
#### 3種類全ての目的変数を使用したIrisのモデルを作成
Irisデータセットのtrain.csvの中で、目的変数Speciesに含まれる3種類全てを分類できるモデルを作成してください。


[Iris Species](https://www.kaggle.com/uciml/iris/data)


2クラスの分類と3クラス以上の分類の違いを考慮してください。それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。


**《ヒント》**


以下の2箇所は2クラス分類特有の処理です。

```python
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
```

```python
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
```

メソッドは以下のように公式ドキュメントを確認してください。


[tf.nn.sigmoid_cross_entropy_with_logits  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/nn/sigmoid_cross_entropy_with_logits)


[tf.math.sign  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/math/sign)


＊tf.signとtf.math.signは同じ働きをします。

In [77]:
df = pd.read_csv(dataset_path)
display(df)
X = np.array(df.iloc[:, 1:5])
y = np.array(pd.get_dummies(df.iloc[:, 5]))
display(X.shape)
display(y.shape)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


(150, 4)

(150, 3)

In [78]:
y = y.astype(np.int)
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [114]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1], 0, 0.1)),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2], 0, 0.1)),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes], 0, 0.1))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1], 0, 0.1)),
        'b2': tf.Variable(tf.random_normal([n_hidden2], 0, 0.1)),
        'b3': tf.Variable(tf.random_normal([n_classes], 0, 0.1))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    layer_output = tf.nn.softmax(layer_output)
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = -tf.reduce_sum(Y*tf.log(logits+1e-12))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.argmax(Y, axis=1), tf.argmax(logits, axis=1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_val_loss = 0
        total_val_acc = 0
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            _, loss, acc = sess.run([train_op, loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
            total_val_loss += val_loss
            total_val_acc += val_acc
            total_loss += loss
            total_acc += acc
        total_loss /= total_batch
        total_acc /= total_batch
        total_val_loss /= total_batch
        total_val_acc /= total_batch
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 5.7865, val_loss : 15.6580, acc : 0.333, val_acc : 0.708
Epoch 1, loss : 2.6775, val_loss : 11.4094, acc : 1.000, val_acc : 0.625
Epoch 2, loss : 1.7245, val_loss : 10.0576, acc : 1.000, val_acc : 0.708
Epoch 3, loss : 0.6799, val_loss : 11.3212, acc : 1.000, val_acc : 0.708
Epoch 4, loss : 0.5415, val_loss : 15.7855, acc : 1.000, val_acc : 0.667
Epoch 5, loss : 0.4387, val_loss : 9.9209, acc : 1.000, val_acc : 0.750
Epoch 6, loss : 1.0310, val_loss : 5.4705, acc : 1.000, val_acc : 0.917
Epoch 7, loss : 0.5486, val_loss : 6.8861, acc : 1.000, val_acc : 0.917
Epoch 8, loss : 0.1615, val_loss : 4.4847, acc : 1.000, val_acc : 0.917
Epoch 9, loss : 0.1103, val_loss : 4.3488, acc : 1.000, val_acc : 0.917
test_acc : 1.000


### 【問題4】
#### House Pricesのモデルを作成
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。


[House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)


この中の`train.csv`をダウンロードし、目的変数として`SalePrice`、説明変数として、`GrLivArea`と`YearBuilt`を使ってください。説明変数はさらに増やしても構いません。


分類問題と回帰問題の違いを考慮してください。



### 【問題5】
#### MNISTのモデルを作成
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。


3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。


スクラッチで実装したモデルの再現を目指してください。